## Import_Modules

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date
import os.path, time
import csv
import re
import selenium
from selenium import webdriver
from fake_useragent import UserAgent

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests_cache
session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'my-program/1.0'

## Connect_functions
1. Request
2. BS settings

In [3]:
def get_html(url, params=None):
    user =UserAgent().random
    HEADERS = {
    'User-Agent': user}
    req = requests.get(url, headers=HEADERS, params=params)
    return req

In [4]:
def get_content(html):
    soup = BeautifulSoup(html, 'lxml')
    ''' Seach '''
    try:
        profile_data = soup.find('table').find_all('span')
        data = []
        for tag in profile_data:
            data.append(tag.text.strip(' '))

        return data
    except:
        pass
    finally:
        pass

## Seacrh_by_conditions
1. Choose periods & Sectors
2. Check in SPB
3. Finviz

In [5]:
industry_list = ['ind_creditservices','ind_insurancelife','ind_insurancepropertycasualty','ind_insurancereinsurance','ind_insurancespecialty','ind_insurancebrokers','ind_diagnosticsresearch','ind_medicaldevices','ind_medicaldistribution','ind_medicalinstrumentssupplies','ind_communicationequipment','ind_computerhardware','ind_consumerelectronics','ind_electroniccomponents','ind_electronicscomputerdistribution','ind_semiconductorequipmentmaterials','ind_semiconductors','ind_solar','ind_electricalequipmentparts']
sector_list = ['sec_communicationservices','sec_consumercyclical','sec_consumerdefensive','sec_industrials','sec_realestate',]
all_industry_sectors = industry_list+sector_list


Year = r'https://finviz.com/screener.ashx?v=171&f=cap_smallover,geo_usa,!!,ta_perf_52w30u&ft=3'
Half = r'https://finviz.com/screener.ashx?v=171&f=cap_smallover,geo_usa,!!,ta_perf_26w30u&ft=4'
Quartal = r'https://finviz.com/screener.ashx?v=171&f=cap_smallover,geo_usa,!!,ta_perf_13w30u&ft=4'
Month = r'https://finviz.com/screener.ashx?v=171&f=cap_smallover,geo_usa,!!,ta_perf_4w30u&ft=4'
len(all_industry_sectors)

24

In [6]:
def find_in_spb(x):
    with open(r"C:\Users\nickl\PycharmProjects\Market\Listing_SPB.csv",encoding="utf8", errors='ignore', newline='') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=';')
        for line in reader:
            er = (line['Tiker'])
            if er == x:
                return x

In [7]:
def fin_viz_check_syndicate(fin_viz_synd): # 1 url
    fin_vz_urls = []
    
    for syndicate in fin_viz_synd:
     
        fin_vz_urls.append(re.sub('!!',syndicate,Year))
        fin_vz_urls.append(re.sub('!!',syndicate,Half))
        fin_vz_urls.append(re.sub('!!',syndicate,Quartal))
        fin_vz_urls.append(re.sub('!!',syndicate,Month))
    watch_list = []
    for comp in fin_vz_urls:
        if check_fin_url(comp) ==False:
            pass
        else:
            watch_list.extend(check_fin_url(comp))
    len(watch_list)
    return watch_list

In [8]:
def check_fin_url(fin_viz_url):  
    
    fiv_fiz_tech_req = get_html(fin_viz_url).text
    soup_viz = BeautifulSoup(fiv_fiz_tech_req,'lxml')
    head_off = soup_viz.find('div',id='screener-content')


   
    if head_off.find_all('tr',valign='top')!=[]: # если есть строки, то начинает парсить
        down_list_corporates = []
        
        count_rows = head_off.find_all('tr',valign='top')
    
        for i in count_rows:
            if find_in_spb(i.find('a',class_="screener-link-primary").text):
                
                comp = [i.find('a',class_="screener-link-primary").text,
                       i.find_all('a',class_="screener-link")[6].text,]
        
                down_list_corporates.append(comp)
    
        return(down_list_corporates)
    else:
        return False

In [9]:
def make_a_watch_list():
    find_falling_companies = fin_viz_check_syndicate(all_industry_sectors)  
 
    data_falling_companies = pd.DataFrame(find_falling_companies,columns=['Tiker','Down'])
    find_all_tikers_by_falling_companies = set([i[0] for i in find_falling_companies])
    print(len(find_all_tikers_by_falling_companies))
    ouput_list = list(map(find_in_spb,find_all_tikers_by_falling_companies))
    return ouput_list
w_l = make_a_watch_list()

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\fake_useragent\utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "C:\ProgramData\Anaconda3\lib\site-packages\fake_useragent\utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


60


## Informations
1. Find Report date
2. General info about Company

In [10]:
def print_full_about_company(data, tiker) -> object:
    last_report = datetime.strptime((data[((data.index('Next Report')) + 1)].strip(' ')), '%m/%d/%Y')
    next_data = datetime.strptime((data[((data.index('Last Report')) + 1)].strip(' ')), '%m/%d/%Y')
    
    
    if next_data < datetime.today():
        next_data = last_report
    else:
        next_data = datetime.today()
    days = next_data - datetime.today()

    full_about_company = []
    full_about_company.append(tiker)

    full_about_company.append(next_data.strftime("%m/%d/%Y"))

    full_about_company.append('Осталось ' + str(days.days+1) + ' дней')

    return full_about_company

In [11]:
def date_before(tiker):
    actual_tiker = tiker.upper()
    new_url = f'https://www.wsj.com/market-data/quotes/{actual_tiker}/financials'
    html = get_html(new_url)
    if html.status_code == 200:
        r = get_content(html.text)
        date_comp = print_full_about_company(r,actual_tiker)
    return date_comp

In [20]:
def info_about_company_from_yf(tiker):
    query_comp = yf.Ticker(tiker.upper(),session=session)
    info = query_comp.info # return all like Json
    input_spisok = ['sector','currentPrice','fullTimeEmployees','longBusinessSummary','website','longName','logo_url']
    ouput_info_spisok = []
  
    for inform in input_spisok:
        try:
            ouput_info_spisok.append(info[inform])
        except:
            ouput_info_spisok.append('None')
        finally:
            pass

    return ouput_info_spisok

In [21]:
def display_info(from_finviz_list):
    return [date_before(tiker)+info_about_company_from_yf(tiker)for tiker in from_finviz_list]
        
    

In [23]:
data = display_info(w_l)

In [24]:
row_watch_list = (
    pd.DataFrame
    (data,columns = ['Tiker','Report_Date','Days_before','sector','currentPrice','fullTimeEmployees','longBusinessSummary','website','longName','logo_url'])
                 )

## Work_with_DF
1. DF Dispaly settings
2. Make an excel file

In [25]:
def make_pretty_df(df):    
    
    
    pd.set_option('max_colwidth', None)
    dfStyler = df.style.set_properties(**{'text-align': 'left'})
    dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
    return df


In [27]:
def convert_to_excel(new_watch_list):
    writer = pd.ExcelWriter(f'Watch list.{datetime.today().strftime("%m_%d_%Y")}.xlsx')
    new_watch_list.to_excel(writer)
# save the excel
    writer.save()
    print('DataFrame is written successfully to Excel File.')

In [28]:
convert_to_excel(row_watch_list)

DataFrame is written successfully to Excel File.


## Look_at_the_OutPut_File

In [29]:
pd.set_option('max_colwidth', None)
frame = pd.read_excel(f'Watch list.{datetime.today().strftime("%m_%d_%Y")}.xlsx',parse_dates=['Report_Date'])

In [30]:
frame.sort_values(by = 'Report_Date')

,Unnamed: 0,Tiker,Report_Date,Days_before,sector,currentPrice,fullTimeEmployees,longBusinessSummary,website,longName,logo_url
32,32,CVGW,2021-11-22,Осталось 1 дней,Consumer Defensive,42.07,3971,"Calavo Growers, Inc. markets and distributes avocados, prepared avocados, and other perishable foods to retail grocery and foodservice customers, club stores, mass merchandisers, food distributors, and wholesale customers worldwide. It operates in three segments: Fresh Products, Calavo Foods, and Renaissance Food Group (RFG). The Fresh products segment distributes avocados and other fresh produce products; and procures avocados grown in California, Mexico, and Peru, as well as various other commodities, including tomatoes and papayas. The Calavo Foods segment is involved in purchasing, manufacturing, and distributing prepared avocado products, including guacamole and salsa. The RFG segment manufactures and distributes fresh-cut fruits and vegetables, fresh prepared entrÃ©e salads, wraps, sandwiches, and fresh snacking products, as well as ready-to-heat entrees, other hot bar and various deli items, meals kits and related components, and salad kits. The company offers its products under the Calavo and RFG brands, and related logos; and Avo Fresco, Bueno, Calavo Gold, Calavo Salsa Lisa, Salsa Lisa, Celebrate the Taste, El Dorado, Fresh Ripe, Select, Taste of Paradise, The First Name in Avocados, Tico, Mfresh, Maui Fresh International, Triggered Avocados, ProRipeVIP, RIPE NOW!, Garden Highway Fresh Cut, Garden Highway, and Garden Highway Chef Essentials trademarks. Calavo Growers, Inc. was founded in 1924 and is headquartered in Santa Paula, California.",http://www.calavo.com,"Calavo Growers, Inc.",https://logo.clearbit.com/calavo.com
2,2,ATI,2022-01-25,Осталось 64 дней,Industrials,16.235,6500,"Allegheny Technologies Incorporated manufactures and sells specialty materials and components worldwide. The company operates in two segments, High Performance Materials & Components and Advanced Alloys & Solutions. The company produces high performance materials, including titanium and titanium-based alloys; nickel-and cobalt-based alloys and superalloys; zirconium and related alloys, such as hafnium and niobium; powder alloys; and other specialty materials in long product forms of ingots, billets, bars, rods, wires, and shapes and rectangles, as well as seamless tubes, plus precision forgings, castings, components, and machined parts to the aerospace and defense, medical, oil and gas, and electrical energy markets. It also provides stainless steel, nickel-based alloys, specialty alloys, and titanium and titanium-based alloys in various product forms, including plate, sheet, and precision rolled strip products to various markets, such as chemical and hydrocarbon processing,. Allegheny Technologies Incorporated was founded in 1960 and is based in Pittsburgh, Pennsylvania.",http://www.atimetals.com,Allegheny Technologies Incorporated,https://logo.clearbit.com/atimetals.com
42,42,LVS,2022-01-26,Осталось 65 дней,Consumer Cyclical,38.55,44500,"Las Vegas Sands Corp., together with its subsidiaries, develops, owns, and operates integrated resorts in Asia and the United States. It owns and operates The Venetian Macao Resort Hotel, the Londoner Macao, The Parisian Macao, The Plaza Macao and Four Seasons Hotel Macao, Cotai Strip, and the Sands Macao in Macao, the People's Republic of China; and Marina Bay Sands in Singapore. The company also owns and operates The Venetian Resort Hotel Casino on the Las Vegas Strip; and the Sands Expo and Convention Center in Las Vegas, Nevada. Its integrated resorts feature accommodations, gaming, entertainment and retail malls, convention and exhibition facilities, celebrity chef restaurants, and other amenities. Las Vegas Sands Corp. was founded in 1988 and is based in Las Vegas, Nevada.",http://www.sands.com,Las Vegas Sands Corp.,https://logo.clearbit.com/sands.com
48,48,ADS,2022-01-27,Осталось 66 дней,Financial Services